In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

import joblib

2026-02-27 09:41:35.494931: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-27 09:41:35.582953: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-27 09:41:37.309207: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
PROJECT_PATH = '../'

In [3]:
WINDOW_SIZE = 30
FORECAST_HORIZON = 5

In [4]:
df = pd.read_csv(PROJECT_PATH + '/LSTM_Ready_Dataset_Old.csv')
df

,timestamp,node_cpu_usage_10.35.29.108:10250,node_cpu_usage_10.35.29.109:10250,node_cpu_usage_10.35.29.110:10250,node_mem_usage_10.35.29.108:10250,node_mem_usage_10.35.29.109:10250,node_mem_usage_10.35.29.110:10250,node_cpu_req_unknown,node_cpu_req_aj-aung-k8s-master,node_cpu_req_aj-aung-k8s-worker1,...,node_mem_req_aj-aung-k8s-master,node_mem_req_aj-aung-k8s-worker1,node_mem_req_aj-aung-k8s-worker2,node_cpu_cap_aj-aung-k8s-master,node_cpu_cap_aj-aung-k8s-worker1,node_cpu_cap_aj-aung-k8s-worker2,node_mem_cap_aj-aung-k8s-master,node_mem_cap_aj-aung-k8s-worker1,node_mem_cap_aj-aung-k8s-worker2,cluster_pods_pending
0,2024-01-01 00:00:00,0.208623,5.975408,0.000000,1922273280,2561769472,0.0,0.2,1.2,3.95,...,461373440.0,2.638217e+09,0.000000e+00,4.0,4.0,0.0,8.332218e+09,8.332227e+09,0.000000e+00,2.0
1,2024-01-01 00:01:00,0.326459,5.980638,0.000000,1930141696,2591625216,0.0,0.8,1.2,3.95,...,461373440.0,2.638217e+09,0.000000e+00,4.0,4.0,0.0,8.332218e+09,8.332227e+09,0.000000e+00,5.0
2,2024-01-01 00:02:00,0.219911,5.765409,1.515837,1926356992,2586947584,374190080.0,0.0,1.2,3.95,...,461373440.0,2.638217e+09,1.149239e+09,4.0,4.0,4.0,8.332218e+09,8.332227e+09,7.289938e+09,0.0
3,2024-01-01 00:03:00,0.221737,5.879989,2.850323,1940553728,2593882112,388370432.0,0.0,1.2,3.95,...,461373440.0,2.638217e+09,1.417675e+09,4.0,4.0,4.0,8.332218e+09,8.332227e+09,7.289938e+09,0.0
4,2024-01-01 00:04:00,0.234354,5.842184,4.132435,1887465472,2598359040,400961536.0,0.0,1.2,3.95,...,461373440.0,2.638217e+09,1.820328e+09,4.0,4.0,4.0,8.332218e+09,8.332227e+09,7.289938e+09,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19477,2024-01-14 12:37:00,0.215379,5.160844,5.077805,1995857920,2587627520,409989120.0,0.0,1.2,3.35,...,461373440.0,2.235564e+09,2.222981e+09,4.0,4.0,4.0,8.332218e+09,8.332227e+09,7.195566e+09,0.0
19478,2024-01-14 12:38:00,0.200130,5.118348,5.014877,1952833536,2601111552,412585984.0,0.0,1.2,3.35,...,461373440.0,2.235564e+09,2.222981e+09,4.0,4.0,4.0,8.332218e+09,8.332227e+09,7.195566e+09,0.0
19479,2024-01-14 12:39:00,0.209720,5.174387,5.084887,1955295232,2584264704,415567872.0,0.0,1.2,3.35,...,461373440.0,2.235564e+09,2.222981e+09,4.0,4.0,4.0,8.332218e+09,8.332227e+09,7.195566e+09,0.0
19480,2024-01-14 12:40:00,0.202781,5.333707,5.059837,1951297536,2554994688,416882688.0,0.0,1.2,3.35,...,461373440.0,2.235564e+09,2.222981e+09,4.0,4.0,4.0,8.332218e+09,8.332227e+09,7.195566e+09,0.0


In [5]:
df.index = pd.to_datetime(df['timestamp'], format = '%Y-%m-%d %H:%M:%S')
df

,timestamp,node_cpu_usage_10.35.29.108:10250,node_cpu_usage_10.35.29.109:10250,node_cpu_usage_10.35.29.110:10250,node_mem_usage_10.35.29.108:10250,node_mem_usage_10.35.29.109:10250,node_mem_usage_10.35.29.110:10250,node_cpu_req_unknown,node_cpu_req_aj-aung-k8s-master,node_cpu_req_aj-aung-k8s-worker1,...,node_mem_req_aj-aung-k8s-master,node_mem_req_aj-aung-k8s-worker1,node_mem_req_aj-aung-k8s-worker2,node_cpu_cap_aj-aung-k8s-master,node_cpu_cap_aj-aung-k8s-worker1,node_cpu_cap_aj-aung-k8s-worker2,node_mem_cap_aj-aung-k8s-master,node_mem_cap_aj-aung-k8s-worker1,node_mem_cap_aj-aung-k8s-worker2,cluster_pods_pending
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:00:00,2024-01-01 00:00:00,0.208623,5.975408,0.000000,1922273280,2561769472,0.0,0.2,1.2,3.95,...,461373440.0,2.638217e+09,0.000000e+00,4.0,4.0,0.0,8.332218e+09,8.332227e+09,0.000000e+00,2.0
2024-01-01 00:01:00,2024-01-01 00:01:00,0.326459,5.980638,0.000000,1930141696,2591625216,0.0,0.8,1.2,3.95,...,461373440.0,2.638217e+09,0.000000e+00,4.0,4.0,0.0,8.332218e+09,8.332227e+09,0.000000e+00,5.0
2024-01-01 00:02:00,2024-01-01 00:02:00,0.219911,5.765409,1.515837,1926356992,2586947584,374190080.0,0.0,1.2,3.95,...,461373440.0,2.638217e+09,1.149239e+09,4.0,4.0,4.0,8.332218e+09,8.332227e+09,7.289938e+09,0.0
2024-01-01 00:03:00,2024-01-01 00:03:00,0.221737,5.879989,2.850323,1940553728,2593882112,388370432.0,0.0,1.2,3.95,...,461373440.0,2.638217e+09,1.417675e+09,4.0,4.0,4.0,8.332218e+09,8.332227e+09,7.289938e+09,0.0
2024-01-01 00:04:00,2024-01-01 00:04:00,0.234354,5.842184,4.132435,1887465472,2598359040,400961536.0,0.0,1.2,3.95,...,461373440.0,2.638217e+09,1.820328e+09,4.0,4.0,4.0,8.332218e+09,8.332227e+09,7.289938e+09,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-14 12:37:00,2024-01-14 12:37:00,0.215379,5.160844,5.077805,1995857920,2587627520,409989120.0,0.0,1.2,3.35,...,461373440.0,2.235564e+09,2.222981e+09,4.0,4.0,4.0,8.332218e+09,8.332227e+09,7.195566e+09,0.0
2024-01-14 12:38:00,2024-01-14 12:38:00,0.200130,5.118348,5.014877,1952833536,2601111552,412585984.0,0.0,1.2,3.35,...,461373440.0,2.235564e+09,2.222981e+09,4.0,4.0,4.0,8.332218e+09,8.332227e+09,7.195566e+09,0.0
2024-01-14 12:39:00,2024-01-14 12:39:00,0.209720,5.174387,5.084887,1955295232,2584264704,415567872.0,0.0,1.2,3.35,...,461373440.0,2.235564e+09,2.222981e+09,4.0,4.0,4.0,8.332218e+09,8.332227e+09,7.195566e+09,0.0


In [6]:
# =====================================================================
# SETUP & DATA LOADING
# =====================================================================

cluster_cpu_req = pd.DataFrame(
    {'cluster_cpu_req': df['node_cpu_req_aj-aung-k8s-worker1'] + df['node_cpu_req_aj-aung-k8s-worker2']}
)

cluster_cpu_cap = pd.DataFrame(
    {'cluster_cpu_cap': df['node_cpu_cap_aj-aung-k8s-worker1'] + df['node_cpu_cap_aj-aung-k8s-worker2']}
)

cluster_mem_req = pd.DataFrame(
    {'cluster_mem_req': df['node_mem_req_aj-aung-k8s-worker1'] + df['node_mem_req_aj-aung-k8s-worker2']}
)

cluster_mem_cap = pd.DataFrame(
    {'cluster_mem_cap': df['node_mem_cap_aj-aung-k8s-worker1'] + df['node_mem_cap_aj-aung-k8s-worker2']}
)

features = pd.concat([cluster_cpu_req, cluster_cpu_cap, cluster_mem_req, cluster_mem_cap, df['cluster_pods_pending']], axis = 1)

target = cluster_cpu_req

In [7]:
features.info()
features

<class 'pandas.DataFrame'>
DatetimeIndex: 19482 entries, 2024-01-01 00:00:00 to 2024-01-14 12:41:00
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   cluster_cpu_req       19482 non-null  float64
 1   cluster_cpu_cap       19482 non-null  float64
 2   cluster_mem_req       19482 non-null  float64
 3   cluster_mem_cap       19482 non-null  float64
 4   cluster_pods_pending  19482 non-null  float64
dtypes: float64(5)
memory usage: 913.2 KB


,cluster_cpu_req,cluster_cpu_cap,cluster_mem_req,cluster_mem_cap,cluster_pods_pending
timestamp,,,,,
2024-01-01 00:00:00,3.95,4.0,2.638217e+09,8.332227e+09,2.0
2024-01-01 00:01:00,3.95,4.0,2.638217e+09,8.332227e+09,5.0
2024-01-01 00:02:00,5.60,8.0,3.787457e+09,1.562216e+10,0.0
2024-01-01 00:03:00,6.00,8.0,4.055892e+09,1.562216e+10,0.0
2024-01-01 00:04:00,6.60,8.0,4.458545e+09,1.562216e+10,0.0
...,...,...,...,...,...
2024-01-14 12:37:00,6.60,8.0,4.458545e+09,1.552779e+10,0.0
2024-01-14 12:38:00,6.60,8.0,4.458545e+09,1.552779e+10,0.0
2024-01-14 12:39:00,6.60,8.0,4.458545e+09,1.552779e+10,0.0


In [8]:
features.dtypes

cluster_cpu_req         float64
cluster_cpu_cap         float64
cluster_mem_req         float64
cluster_mem_cap         float64
cluster_pods_pending    float64
dtype: object

In [9]:
target.info()
target

<class 'pandas.DataFrame'>
DatetimeIndex: 19482 entries, 2024-01-01 00:00:00 to 2024-01-14 12:41:00
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cluster_cpu_req  19482 non-null  float64
dtypes: float64(1)
memory usage: 304.4 KB


,cluster_cpu_req
timestamp,
2024-01-01 00:00:00,3.95
2024-01-01 00:01:00,3.95
2024-01-01 00:02:00,5.60
2024-01-01 00:03:00,6.00
2024-01-01 00:04:00,6.60
...,...
2024-01-14 12:37:00,6.60
2024-01-14 12:38:00,6.60
2024-01-14 12:39:00,6.60


In [10]:
target.dtypes

cluster_cpu_req    float64
dtype: object

In [12]:
features.min()

cluster_cpu_req         0.0
cluster_cpu_cap         0.0
cluster_mem_req         0.0
cluster_mem_cap         0.0
cluster_pods_pending    0.0
dtype: float64

In [ ]:
# =====================================================================
# CHRONOLOGICAL SPLIT (70/15/15)
# =====================================================================

n = len(features)

train_idx = int(n * 0.7)
val_idx   = int(n * 0.85)

# Split features (for X)
X_train_raw = features[:train_idx]
X_val_raw   = features[train_idx:val_idx]
X_test_raw  = features[val_idx:]

# Split target (for y)
y_train_raw = target[:train_idx]
y_val_raw   = target[train_idx:val_idx]
y_test_raw  = target[val_idx:]

X_train_raw.shape, X_val_raw.shape, X_test_raw.shape, y_train_raw.shape, y_val_raw.shape, y_test_raw.shape

In [ ]:
# =====================================================================
# DUAL SCALING (CRITICAL STEP)
# =====================================================================

scaler_inputs  = MinMaxScaler(feature_range = (0, 1))
X_train_scaled = scaler_inputs.fit_transform(X_train_raw)
X_val_scaled   = scaler_inputs.transform(X_val_raw)
X_test_scaled  = scaler_inputs.transform(X_test_raw)

scaler_target  = MinMaxScaler(feature_range = (0, 1))
y_train_scaled = scaler_target.fit_transform(y_train_raw)
y_val_scaled   = scaler_target.transform(y_val_raw)
y_test_scaled  = scaler_target.transform(y_test_raw)

joblib.dump(scaler_inputs, 'Multi-Var_Scaler_Inputs.pkl')
joblib.dump(scaler_target, 'Multi-Var_Scaler_Target.pkl')
print("Success: 'Multi-Var_Scaler_Inputs.pkl' & 'Multi-Var_Scaler_Target.pkl' saved.")

In [ ]:
# =====================================================================
# SLIDING WINDOW (MULTIVARIATE)
# =====================================================================

def multivariate_data(dataset, target, start_index, end_index, history_size, target_size):
	data   = []
	labels = []

	start_index = start_index + history_size
	if end_index is None:
		end_index = len(dataset) - target_size

	for i in range(start_index, end_index):
		indices = range(i - history_size, i)
		data.append(dataset[indices])
		labels.append(target[i + target_size])

	return np.array(data), np.array(labels)

X_train, y_train = multivariate_data(X_train_scaled, y_train_scaled, 0, None, WINDOW_SIZE, FORECAST_HORIZON)
X_val  , y_val   = multivariate_data(X_val_scaled  , y_val_scaled  , 0, None, WINDOW_SIZE, FORECAST_HORIZON)
X_test , y_test  = multivariate_data(X_test_scaled , y_test_scaled , 0, None, WINDOW_SIZE, FORECAST_HORIZON)

print(f"Train Shape: {X_train.shape}")
print(f"Target Shape: {y_train.shape}")

In [ ]:
X_train.shape[1], X_train.shape[2]

In [ ]:
model = Sequential()
model.add(LSTM(units = 256, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units = 256, input_shape = (X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss = 'mae', optimizer = 'adam')
model.summary()
history = model.fit(X_train, y_train, epochs = 100, batch_size = 32, validation_data = (X_val, y_val), verbose = 1)

In [ ]:
plt.figure(figsize = (25, 10))
plt.plot(history.history['loss'], label = 'Train Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.legend()
plt.show()

In [ ]:
# =====================================================================
# EVALUATION & PLOTTING
# =====================================================================

y_scaled_predicted = model.predict(X_test)

y_predicted_converted_back = scaler_target.inverse_transform(y_scaled_predicted)
y_actuals                  = scaler_target.inverse_transform(y_test.reshape(-1, 1))

rmse = np.sqrt(mean_squared_error(y_actuals, y_predicted_converted_back))
mae = mean_absolute_error(y_actuals, y_predicted_converted_back)
r2 = r2_score(y_actuals, y_predicted_converted_back)
mape = mean_absolute_percentage_error(y_actuals, y_predicted_converted_back) * 100

print("\n--- OFFICIAL AUTOSCALER PERFORMANCE ---")
print(f"Test RMSE: {rmse:.2f} vCores")
print(f"Test MAE:  {mae:.2f} vCores")
print(f"On average, the model's 5-minute forecast is off by {mae:.2f} vCores.")
print(f"R-squared Score: {r2:.4f} (Model explains {r2 * 100:.2f}% of the variance)")
print(f"MAPE: {mape:.2f}% (Predictions are off by an average of {mape:.2f}%)")

In [ ]:
plt.figure(figsize = (25, 10))
plt.plot(history.history['loss'], label = 'Training Loss (MSE)', color = 'blue')
plt.plot(history.history['val_loss'], label = 'Validation Loss (MSE)', color = 'orange')
plt.title('LSTM Learning Curve: Training Loss vs. Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (Mean Squared Error)')
plt.legend(loc = 'upper right')
plt.grid(True, linestyle = '--', alpha = 0.7)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (25, 10))
plt.plot(y_predicted_converted_back, label = 'Actual vCores', color = 'blue', alpha = 0.6)
plt.plot(y_actuals, label = 'Predicted vCores (5m ahead)', color='red', linestyle = '--', alpha = 0.8)
plt.title('Test Data: Actual CPU Requests vs Predicted CPU Requests')
plt.xlabel('Time Steps')
plt.ylabel('vCores')
plt.legend(loc = 'upper right')
plt.grid(True, linestyle = '--', alpha = 0.5)
plt.tight_layout()
plt.show()

In [ ]:
model.save('Multi-Variable_LSTM_Model.keras')
print(f"Success: Model fully saved to Multi-Variable_LSTM_Model.keras")

In [ ]:
allowed_error = y_actuals * 0.10  # 10% margin of error
absolute_errors = np.abs(y_actuals - y_predicted_converted_back)
correct_predictions = np.sum(absolute_errors <= allowed_error)

custom_accuracy = (correct_predictions / len(y_actuals)) * 100
print(f"Threshold Accuracy: {custom_accuracy:.2f}% of predictions were within a 10% margin of error.")